In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import os
os.chdir('/content/drive/MyDrive/ADM_HW_3')

In [10]:
from bs4 import BeautifulSoup as bs
from urllib.request import Request, urlopen
from urllib.parse import urlparse
from tqdm import tqdm
from datetime import datetime
import requests
import re
import os
import time
import pickle

# 1. Data Collection

DO NOT RUN the following code.


## 1.1 Get the list of places

Due to the HTTP request timeout, we'll split the requests into two chunks

Chunk 1 [Page 1- Page 200]

In [ ]:
with open('urls_1.txt', 'w') as f:

  for i in tqdm(range(1,201)):
    r = "https://www.atlasobscura.com/places?page="+str(i)+"&sort=likes_count"
    response = requests.get(r, headers = {'User-agent': 'your bot 0.1'})
    #response = requests.get(r)
    soup = bs(response.text)

    for link in soup.find_all('a', class_='content-card content-card-place'):
        #urls.append(link.get('href')))
        f.write('https://www.atlasobscura.com'+link.get('href')+'\n')

f.close
  

100%|██████████| 200/200 [01:31<00:00,  2.17it/s]


<function TextIOWrapper.close()>

Chunk 2 [Page 201- Page 400]

In [ ]:
with open('urls_2.txt', 'w') as f:
#next step is to iterate this process over the 400 pages
  for i in tqdm(range(201,401)):
    r = "https://www.atlasobscura.com/places?page="+str(i)+"&sort=likes_count"
    response = requests.get(r, headers = {'User-agent': 'your bot 0.1'})
    #response = requests.get(r)
    soup = bs(response.text)

    for link in soup.find_all('a', class_='content-card content-card-place'):
        #urls.append(link.get('href')))
        f.write('https://www.atlasobscura.com'+link.get('href')+'\n')

f.close
  

100%|██████████| 200/200 [01:35<00:00,  2.10it/s]


<function TextIOWrapper.close()>

In [ ]:
#merging the two files into a single file
!cat urls_1.txt urls_2.txt > urlsv1.txt

#!rm urls_1.txt
#!rm urls_2.txt

## 1.2 Crawl places

DO NOT RUN THESE CELLS.

In [ ]:
#write the complete ulrs in our txt file
out = open('places.txt', 'w')
input = open('urlsv1.txt', 'r')
lines = input.readlines()

for line in lines:
  out.write('https://www.atlasobscura.com'+line)

input.close()
out.close()

In [ ]:
#create the folder in which we will store all pages' folders
os.mkdir('/content/drive/MyDrive/ADM_HW_3/html_download')

In [ ]:
from urllib.request import urlopen
from urllib.error import HTTPError
import shutil
import time

checkpoints = []

my_file = open('/content/drive/MyDrive/ADM_HW_3/places.txt', 'r')
lines = my_file.readlines()
#flag = 0
for i in tqdm(range(0,400)):
  if os.path.isdir('./html_download/page%s' % (i+1)) == False:
    os.mkdir('./html_download/page%s' % (i+1))

  for line, j in zip(lines[i*18:(i*18)+18], range(1,19)):
    try:
      if os.path.isfile('./html_download/page%s/HTML%s.html'%((i+1),j)) == False:
        page = urlopen(line)
        f = open('./html_download/page%s/HTML%s.html'%((i+1),j), 'w')
        soup = bs(page)
        f.write(soup.prettify())
        f.close()
    except HTTPError as err:
      #when the the website blocks us, save the position of the page we were in
      checkpoints.append(i) #checkpoint list to retrieve missed information in future
      #flag=i

my_file.close()

100%|██████████| 401/401 [17:27<00:00,  2.61s/it]


In [ ]:
#check missing htlms
for i in range(1,401):
  if len(os.listdir('./html_download/page%s' %i)) < 18:
    print('Page%s:'%i,len(os.listdir('./html_download/page%s' %i)))

Since the site blocked us while we were trying to retrieve all html files, we stored the page number correspoing at the moment we were blocked.
By doing this operation we know the number of page for resume the operation.

## 1.3 Parse Dowloaded Pages

In [ ]:
#function for extracting the information we need from one HTML file.
#We decided to give a soup object to the function, instead of the html directly. 
def extract_single_place(soup):
    
    #1.Place Name
    try:
      name_place = soup.find('h1').getText().replace('\n', ' ').strip()
    except:
      name_place = ' '
    
    
    #2.Place Tags
    try:
      location_tag = soup.find_all('div', class_= 'item-tags col-xs-12')[0].find_all('a')
      #if type(location_tag) == bs4.element.NavigableString:
      location = [t.getText().strip() for t in location_tag]
      #else:
          #location = location_tag.contents[0].text
    except:
      location = ' '
    
    
    #3.num of people who have been there
    try:
      visited = int(soup.find_all('div', {'class': 'title-md item-action-count' })[0].contents[0].replace('\n', ' ').strip())
    except:
      visited = ' '

    
    #4.num of people who want to visit the place
    try:
      wanna_visit = int(soup.find_all('div', {'class': 'title-md item-action-count' })[1].contents[0].replace('\n', ' ').strip())
    except:
      wanna_visit = ' '

  
    #5.Description
    try:
      long_description = [x.text for x in soup.find_all('div', {'id': 'place-body'})]
      desc = re.sub("\s\s+", " ", long_description[0].replace('\n', ' ').strip())
    except:
      desc = ' '

    #6.Short description
    try:
      short_description = soup.find_all('h3', {'class': 'DDPage__header-dek'})[0].contents[0].replace('\n', ' ').strip()
    except:
      short_description = ' '
    

    #7.Nearby Places
    try:
      nearby = []
      nearby.append(soup.find_all('div', class_="DDPageSiderailRecirc__item-title")[0].getText().replace('\n', " ").strip())
      nearby.append(soup.find_all('div', class_="DDPageSiderailRecirc__item-title")[1].getText().replace('\n', " ").strip())
      nearby.append(soup.find_all('div', class_="DDPageSiderailRecirc__item-title")[2].getText().replace('\n', " ").strip())
    except:
      nearby = ' '


    #8.Address of the place
    try:
      tmp = soup.find('address').find('div').contents[0:5:2]
      addr = re.sub('\s\s+', ' ' , ''.join(tmp).replace('\n', ' ').strip())
    except:
      addr = ' '
    


    #9.Altitude and Longitude of the place's location
    try:
      alt_and_lat = soup.find(class_="DDPageSiderail__coordinates js-copy-coordinates").getText().strip().split(',')
      alt = float(alt_and_lat[0])
      lat = float(alt_and_lat[1])
    except:
      alt = ' '
      lat = ' '


    #10.Username of the post editors
    try:
      people = soup.find('div', class_="DDPContributors").find_all('a', attrs={'class':'DDPContributorsList__contributor'})
      contributors = [t.text.replace('\n',' ').strip() for t in people]
    except:
      contributors = ' '
      
    #11. Post Publishing Date
    try:
      string_date = soup.find('div', class_='DDPContributor__name').text.replace('\n', ' ').strip()
      format = '%B %d, %Y'
      public_date = datetime.strptime(string_date,format).date()
    except:
      public_date = ' '

    #12. related lists
    try:
      lists = soup.find_all('div', attrs={'data-gtm-label':'Related Content Card'})[2].find_all('h3',attrs={'class':'Card__heading --content-card-v2-title js-title-content'})
      relatedLists = [l.text.replace('\n', ' ').strip() for l in lists]
    except:
      relatedLists = ' '



    #13. Related places
    try:
      relatedPlaces = []
      places = soup.find_all('div', attrs={'data-gtm-label':'Related Content Card'})[1].find_all('h3',attrs={'class':'Card__heading --content-card-v2-title js-title-content'})
      relatedPlaces = [p.text.replace('\n', ' ').strip() for p in places]
    except:
      relatedPlaces = ' '

    #14. URL Page
    try:
      url = soup.find('link', {'rel':'canonical'})['href']
    except:
      url = ' '

    
    return {'placeName': name_place, 
            'placeTags': location,
            'numPeopleVisited': visited, 
            'numPeopleWant': wanna_visit,
            'placeDesc':desc,
            'placeShortDesc':short_description,
            'placeNearby': nearby,
            'placeAddress' : addr,
            'placeAlt': alt,
            'placeLong' : lat,
            'placeEditors' : contributors,
            'placePubDate': public_date,
            'placeRelatedLists': relatedLists,
            'placeRelatedPlaces': relatedPlaces,
            'placeURL': url}



## Creation of our dataset

In [ ]:
import csv
#now we try to create, for each page folder a page.tsv

for i in tqdm(range(180, 401)):
  os.chdir('/content/drive/MyDrive/ADM_HW_3/html_download/page%s' % i)
  tsv_columns = ['placeName', 'placeTags', 'numPeopleVisited', 'numPeopleWant', 'placeDesc', 'placeShortDesc', 'placeNearby', 'placeAddress', 'placeAlt', 'placeLong', 'placeEditors',  'placePubDate', 'placeRelatedLists', 'placeRelatedPlaces', 'placeURL']

  #list of html files in i-th page folder
  HTML_list = os.listdir('/content/drive/MyDrive/ADM_HW_3/html_download/page%s' % i)

  #tsv_file = 'page.tsv'
  try:
    with open('/content/drive/MyDrive/ADM_HW_3/html_download/page%s/page%i.tsv'%(i,i),'a', newline = '') as tsvfile:
      writer = csv.DictWriter(tsvfile, fieldnames=tsv_columns)
      writer.writeheader()
      for j in range(18):
    
        row = {}
        f = open(HTML_list[j], 'r')
        soup = bs(f, 'html.parser')
        result = extract_single_place(soup)
          #print(result)
            
        for data in result.keys():
            
            row[data] = result[data]
              #print(row)
        writer.writerow(row)
              #else:
                #writer.writerow(' ')

  except IOError:
    print("I/O error")
    f.close
    tsvfile.close

100%|██████████| 221/221 [37:49<00:00, 10.27s/it]


In [ ]:
os.mkdir('/content/drive/MyDrive/ADM_HW_3/html_download')

In [ ]:
#now we merge the files tsv 
with open('tsv_merged.tsv', 'w', newline = '') as tsv_fin:
  writer = csv.DictWriter(tsv_fin, fieldnames= tsv_columns)
  writer.writeheader()
  with open('/content/drive/MyDrive/ADM_HW_3/html_download/page1/page1.tsv', 'r') as tsv1:
    reader = csv.DictReader(tsv1)
    for row in reader: 
      writer.writerow(row)

  files = 0

  for i in tqdm(range(2,401)):
    with open('/content/drive/MyDrive/ADM_HW_3/html_download/page%s/page%s.tsv' %(i,i), 'r', newline='') as tsv:
      files +=1
      reader = csv.DictReader(tsv, fieldnames= tsv_columns)     
      next(reader, None)  # skip the headers
    
      for row in reader:
        #line = tsv_file.readline()
        
        writer.writerow(row)
     
  print(files)
  tsv.close
tsv_fin.close

100%|██████████| 399/399 [00:01<00:00, 225.35it/s]

399


<function TextIOWrapper.close()>

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/ADM_HW_3/tsv_merged.tsv')
df.to_csv('places.tsv', sep = '\t', index = False)

In [ ]:
import pandas as pd
df = pd.read_csv('places.tsv', sep = '\t')

In [ ]:
#checking if there are any duplicates

df[df.duplicated()]

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
666,Frankie's Tiki Room,"['tiki', 'bars', 'Unique Restaurants & Bars', ...",686,1655,No trip would be complete without a drink at a...,The best Tiki bar in Las Vegas.,"['Burlesque Hall of Fame', 'Zak Bagans Haunted...","1712 W Charleston Blvd Las Vegas, Nevada, 8910...",36.1592,-115.1646,"['Dylan', 'shaunk82', 'Molly McBride Jacobson'...",2014-06-07,['17 Hidden Las Vegas Spots It Would Be a Sin ...,"[""Trader Vic's"", 'Mai-Kai Restaurant', 'The To...",https://www.atlasobscura.com/places/frankies-tiki
667,The Lost Caves,"['subterranean', 'underground', 'bars', 'hidde...",168,1654,"Damp, dimly lit, and decorated with skulls, ch...",Hidden in a grotto 26 feet below Nottingham's ...,"[""Watson Fothergill's Office"", 'The Exchange M...","George St Nottingham, England United Kingdom",52.9537,-1.1444,"['Capemarsh', 'erjeffery']",2018-07-02,,"['Caverns Grotto', 'Bell Inn Caves', 'Naples U...",https://www.atlasobscura.com/places/the-lost-c...
668,Bunkers of Carmel,"['bunkers', 'military history', 'military']",666,1648,Anyone looking to get an incredible bird’s-eye...,These secluded and abandoned anti-aircraft def...,"['Park Güell', 'Sant Pau Recinte Modernista', ...",Av. de l'Estatut de Catalunya Barcelona Spain,41.4193,2.1617,"['martiferrer', 'andradeagustinam', 'HereBeDra...",2015-03-31,,"['Krutbanan (Gunpowder Track)', 'Schießplatzhe...",https://www.atlasobscura.com/places/bunkers-de...
669,The Morikami Museum and Japanese Gardens,"['horticulture', 'gardens', 'plants', 'museums']",761,1652,"In 1906, 19-year-old George Morikami immigrate...",Japanese gardens dedicated to the Japanese far...,"['Green Cay Nature Center and Wetlands', 'Bulk...","4000 Morikami Park Road Delray Beach, Florida,...",26.4298,-80.1568,"['TheCandyCaneHustler', 'Helz', 'tobygenest22'...",2013-07-25,['15 Florida Places to Be Nurtured By Nature'],"['Aston Hall', ""Monticello's Vegetable Garden ...",https://www.atlasobscura.com/places/morikami-m...
670,Brazenhead Books,"['bookstores', 'books']",121,1648,Brazenhead Books is a bookstore located inside...,A legendary speakeasy bookstore returns from t...,"['The Conjuring Arts Research Center', 'The Li...","Available on request. New York, New York, 1002...",40.747,-73.986,"['karpantschof', 'lampbane', 'littlebrumble', ...",2016-10-12,['17 Cool Shops in New York'],"['For Keeps Bookstore', 'La Llama', 'Ramalama ...",https://www.atlasobscura.com/places/brazenhead...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7157,V & E Simonetti Historic Tuba Collection,"['music', 'collections', 'museums and collecti...",91,408,Vincent Simonetti began his tuba collection wh...,The humble horn finally finds a place in the sun.,"['The Can Opener', 'Duke Lemur Center', ""Durha...","1825 Chapel Hill Rd Durham, North Carolina Uni...",35.989,-78.9247,"['heypeggy', 'Ivana Viani', 'judybrucef', 'Mol...",2016-10-21,['The Ultimate List of Wonderfully Specific Mu...,"[""St Cecilia's Hall"", 'Museum of Sound', 'Este...",https://www.atlasobscura.com/places/v-e-simone...
7158,Crook Point Bascule Bridge,"['rivers', 'railroads', 'technology', 'bridges...",390,409,"From Providence to East Providence, Rhode Isla...",To avoid the demolition costs this abandoned d...,"['Gingerbread and Pancake Islands', 'Roger Wil...",Gano Park to Crook Point Seekonk River (spanni...,41.8243,-71.3834,"['The Minx', 'Jumboto', 'robbiemaynardcreates']",2015-11-12,,"['Goz Abu Goma Bridge', 'Ponte da Ajuda (Ajuda...",https://www.atlasobscura.com/places/crook-poin...
7159,Camperdown Elm,"['trees', 'parks', 'plants']",225,410,The strangest tree in Brooklyn may be the gnar...,A tree that grows parallel to the ground was d...,"['Cleft Ridge Span', 'The Carousel in Prospect...","Prospect Park Brooklyn, New York, 11225 United...",40.6604,-73.9651,"['Allison', 'Squanjo', 'Yel

In [ ]:
#Clean our dataset
df.drop_duplicates(inplace = True)

In [ ]:
df.to_csv('cleaned_places.tsv', sep = '\t', index = False)

In [ ]:
df = pd.read_csv('cleaned_places.tsv', delimiter = '\t')

In [ ]:
df.head(10)

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...",1837,8637,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"['African Burial Ground National Monument', 'T...","31 Centre St New York, New York, 10007 United ...",40.7134,-74.0046,"['Rebekah Otto', 'Habicurious', 'littlebrumble...",2010-05-08,['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",https://www.atlasobscura.com/places/city-hall-...
1,Highgate Cemetery,"['rivals week', 'vampires', 'hoaxes', 'occult'...",2633,8232,"Opened in 1839, Highgate is one of London’s mo...",London's creepiest cemetery was once the site ...,"[""World's Largest Potted Plant"", 'Dick Whittin...","Swain's Lane, Highgate London, England, N6 Uni...",51.5675,-0.1483,"['gingercinnamon', 'Gavin', 'Martin', 'anthony...",2014-08-09,"[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",https://www.atlasobscura.com/places/highgate-c...
2,Leadenhall Market,"['markets', 'film locations', 'film']",3156,7635,The ornate 19th-century painted roof and cobbl...,This ornate Victorian marketplace was the sett...,"['The Cornhill Devils', ""London's Original and...","London, England, EC3V United Kingdom",51.5126,-0.0834,"['Meg', 'Felth', 'reley250', 'Lyloueen', 'Jule...",2016-08-01,"['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",https://www.atlasobscura.com/places/leadenhall...
3,The Wave Organ,"['sea organ', 'aletrail', 'sounds', 'music', '...",2434,7434,"Located on a jetty in the San Francisco Bay, t...",A huge musical instrument played by the ocean.,"['Long Now Orrery', 'The Stern of the Briganti...","83 Marina Green Dr San Francisco, California, ...",37.8085,-122.4401,"['mbison', 'Chassy', 'Mark Casey', 'Blindcolou...",2008-11-21,"[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",https://www.atlasobscura.com/places/wave-organ
4,Catacombes de Paris,"['ossuaries', 'memento mori', 'catacombs and c...",4467,7075,"In 2004, Parisian police were assigned to do a...","The vast, legendary catacombs hold secrets muc...","['Sculptures de Décure', 'Arago Medallions', ""...","1 Place Denfert-Rochereau Paris, 75014 France",48.8343,2.3322,"['CPilgrim', 'Blindcolour', 'Bhob D', 'Annetta...",2009-02-13,"['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",https://www.atlasobscura.com/places/catacombes...
5,The Evolution Store,"['wonder cabinets', 'purveyors of curiosities'...",2203,6975,Evolution stands out among the clothing stores...,A terrific purveyor of natural history objects...,"[""Merchant's House Museum"", 'The Brown Buildin...","687 Broadway New York, New York, 10012 United ...",40.7281,-73.9948,"['mbison', 'ramosju', 'Yvanec182', 'czhalwaysw...",2008-11-21,"[""The World's Top 100 Wonders in 2018"", ""New Y...","['Kunstkammer Georg Laue', 'Ospedale delle Bam...",https://www.atlasobscura.com/places/evolution-...
6,Magowan's Infinite Mirror Maze,"['optical illusions', 'labyrinths', 'optical o...",909,6513,"People stumble about the labyrinth, arms outst...",A psychedelic labyrinth on the San Francisco bay.,"['Musée Mécanique', 'Jack Early Park', ""Fior D...","Pier 39, Building O-11 San Francisco, Californ...",37.81,-122.4104,"['GuttersnipeRebecca', 'Molly McBride Jacobson...",2016-12-20,"['24 Mesmerizing Mazes and Labyrinths', ""The W...","['The Winchester Mizmaze', 'Cathedral of Thorn...",https://www.atlasobscura.com/places/magowans-i...
7,Underground Tunnels of Los Angeles,"['tunnels', 'subterranean sites', 'architectur...",660,6445,While the rest of the country was forced to go...,"During prohibition,

# 2 Search Engine

### Pre-processing
In this step we will pre-process all informations about the retrieved places

In [6]:
#removing stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
#we define a function for the preprocessing
def pre_processing(txt):

  #remove stopwords
  #with this regular expression we consider only characters, thus we are removing punctuations
  tokens = RegexpTokenizer(r'\w+')   #`RegexpTokenizer`` splits a string into substrings using a regular expression.
  tokens_words = list(tokens.tokenize(txt))

  #creating a set containing all the english stopwords
  stop_words = set(stopwords.words('english'))

  #stemming
  stemmer = PorterStemmer()
  roots = [stemmer.stem(t).strip() for t in tokens_words if t not in stop_words]

  return roots

In [11]:
import pandas as pd

df = pd.read_csv('cleaned_places.tsv', delimiter = '\t')

In [12]:
#apply the preprocessing to the description of the dataset
df['list_words'] = df['placeDesc'].apply(lambda row:pre_processing(row))

In [13]:
df.head(10)

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL,list_words
0,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...",1837,8637,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"['African Burial Ground National Monument', 'T...","31 Centre St New York, New York, 10007 United ...",40.7134,-74.0046,"['Rebekah Otto', 'Habicurious', 'littlebrumble...",2010-05-08,['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",https://www.atlasobscura.com/places/city-hall-...,"[the, first, new, york, citi, subway, built, o..."
1,Highgate Cemetery,"['rivals week', 'vampires', 'hoaxes', 'occult'...",2633,8232,"Opened in 1839, Highgate is one of London’s mo...",London's creepiest cemetery was once the site ...,"[""World's Largest Potted Plant"", 'Dick Whittin...","Swain's Lane, Highgate London, England, N6 Uni...",51.5675,-0.1483,"['gingercinnamon', 'Gavin', 'Martin', 'anthony...",2014-08-09,"[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",https://www.atlasobscura.com/places/highgate-c...,"[open, 1839, highgat, one, london, infam, ceme..."
2,Leadenhall Market,"['markets', 'film locations', 'film']",3156,7635,The ornate 19th-century painted roof and cobbl...,This ornate Victorian marketplace was the sett...,"['The Cornhill Devils', ""London's Original and...","London, England, EC3V United Kingdom",51.5126,-0.0834,"['Meg', 'Felth', 'reley250', 'Lyloueen', 'Jule...",2016-08-01,"['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",https://www.atlasobscura.com/places/leadenhall...,"[the, ornat, 19th, centuri, paint, roof, cobbl..."
3,The Wave Organ,"['sea organ', 'aletrail', 'sounds', 'music', '...",2434,7434,"Located on a jetty in the San Francisco Bay, t...",A huge musical instrument played by the ocean.,"['Long Now Orrery', 'The Stern of the Briganti...","83 Marina Green Dr San Francisco, California, ...",37.8085,-122.4401,"['mbison', 'Chassy', 'Mark Casey', 'Blindcolou...",2008-11-21,"[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",https://www.atlasobscura.com/places/wave-organ,"[locat, jetti, san, francisco, bay, wave, orga..."
4,Catacombes de Paris,"['ossuaries', 'memento mori', 'catacombs and c...",4467,7075,"In 2004, Parisian police were assigned to do a...","The vast, legendary catacombs hold secrets muc...","['Sculptures de Décure', 'Arago Medallions', ""...","1 Place Denfert-Rochereau Paris, 75014 France",48.8343,2.3322,"['CPilgrim', 'Blindcolour', 'Bhob D', 'Annetta...",2009-02-13,"['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",https://www.atlasobscura.com/places/catacombes...,"[in, 2004, parisian, polic, assign, train, exe..."
5,The Evolution Store,"['wonder cabinets', 'purveyors of curiosities'...",2203,6975,Evolution stands out among the clothing stores...,A terrific purveyor of natural history objects...,"[""Merchant's House Museum"", 'The Brown Buildin...","687 Broadway New York, New York, 10012 United ...",40.7281,-73.9948,"['mbison', 'ramosju', 'Yvanec182', 'czhalwaysw...",2008-11-21,"[""The World's Top 100 Wonders in 2018"", ""New Y...","['Kunstkammer Georg Laue', 'Ospedale delle Bam...",https://www.atlasobscura.com/places/evolution-...,"[evolut, stand, among, cloth, store, restaur, ..."
6,Magowan's Infinite Mirror Maze,"['optical illusions', 'labyrinths', 'optical o...",909,6513,"People stumble about the labyrinth, arms outst...",A psychedelic labyrinth on the San Francisco bay.,"['Musée Mécanique', 'Jack Early Park', ""Fior D...","Pier 39, Building O-11 San Francisco, Californ...",37.81,-122.4104,"['GuttersnipeRebecca', 'Molly McBride Jacobson...",2016-12-20,

## 2.1 Conjuctive Query (First search engine)

### 2.1.1. Create your index 

In [ ]:
import itertools

In [ ]:
#store all the words contained in all the descriptions
#concatenate all the lists
all_lists = []
for i in range(len(df['list_words'])):
  all_lists.append(df['list_words'][i])
#create a single list 
all_words = list(itertools.chain.from_iterable(all_lists))


#sorte the words
all_words.sort()

#we want a dictionary without repetitions
words_set = set(all_words)   #for creating the vocabulary

In [ ]:
#create the vocabulary
vocabulary = dict.fromkeys(words_set, 0)

In [ ]:
#create the dictionary mapping all the words with the corresponding item_id
word_dict = {}
item_id = 1
for word in vocabulary.keys():
    word_dict[word] = item_id
    item_id += 1

In [ ]:
#create a pickle file containing the vocabulary
with open('vocabulary.pkl', 'wb') as f:
    pickle.dump(word_dict, f)
f.close()

In [ ]:
#OPEN the pickle file
with open('vocabulary.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)
f.close()

In [ ]:
#creation of the inverted index
inverted_idx = {}
for word, item_id in word_dict.items():
  inverted_idx[item_id] = list(df[df.list_words.apply(lambda row: word in row)].index)     #retrieve the index values of the documents containing that word.

In [ ]:
#create a pickle file containing the inverted index
with open('inverted_index.pkl', 'wb') as f:
    pickle.dump(inverted_idx, f)
f.close()

In [ ]:
#OPEN the pickle file
with open('inverted_index.pkl', 'rb') as f:
    inverted_index = pickle.load(f)
f.close()

### 2.1.2. Execute the query 

In [ ]:
#ask the user the query
query = input('Please insert a query: \n')
query_words = query.split()  #return a list of strings

Please insert a query: 
highgate


In [ ]:
#retrieve a list with the lists of documents containining query words
results = [set(inverted_idx[item]) for item in [word_dict[word] for word in pre_processing(query)]]

In [ ]:
#we want the output docs to contain all the query words: we want the intersection of the set of documents.
intersection = set.intersection(*map(set,results))
list_intersection = list(intersection)

In [ ]:
#retrieve from the original datframe the documents matching the intersection of the query terms
docs = df.filter(items = list_intersection, axis = 0)

In [ ]:
#select the information the user can be interested on
docs = docs[['placeName', 'placeDesc', 'placeURL']]
docs.rename(columns ={'placeName': 'Title', 'placeDesc': 'Description', 'placeURL': 'Url'}, inplace = True)

In [ ]:
docs

,Title,Description,Url
1,Highgate Cemetery,"Opened in 1839, Highgate is one of London’s mo...",https://www.atlasobscura.com/places/highgate-c...
385,Nunhead Cemetery,"Now a Local Nature Reserve, the Nunhead Cemete...",https://www.atlasobscura.com/places/nunhead-ce...
1188,El Sabroso,Amid the over-stuffed wholesale shops and hole...,https://www.atlasobscura.com/places/el-sabroso
166,Kensal Green Cemetery and Catacombs,Kensal Green is the one of the oldest public b...,https://www.atlasobscura.com/places/kensal-gre...
6374,Portland Cement Plant Ruins,These industrial ruins offer an up-close encou...,https://www.atlasobscura.com/places/portland-c...
2888,Plum Island Pink House,"This abandoned house looms over a salt marsh, ...",https://www.atlasobscura.com/places/plum-islan...
2894,Unofficial Lego Toy and Plastic Brick Museum,While most of us see LEGO bricks as a fun past...,https://www.atlasobscura.com/places/toy-and-pl...
1074,Euston's Lost Tunnels,A portal into the London Underground’s past hi...,https://www.atlasobscura.com/places/eustons-lo...
1884,World's Largest Puffin Colony,Iceland is home to around half of all the worl...,https://www.atlasobscura.com/places/worlds-lar...


## 2.2 Conjuctive query and Ranking score (Second search engine)

### 2.2.1) Inverted Index

**TF-IDF** <br>
Term Frequency (TF) is computed as: <br>
$\begin{equation} TF(t,d)=\dfrac{count(t)}{count(d)}= \dfrac{\text{# occurrences of t in d}}{\text{# words in d}} \end{equation}$ <br>
where: <br> 
$t$ interested text <br>
$d$ the document provided. <br>
Inverse Document Frequency (IDF) is computed as:<br>
$\begin{equation} IDF = \log{(\dfrac{\text{# documents}}{\text{# documents t occurs in}})}\end{equation}$ <br>
<br>
Finally:<br> 
$\text{TF-IDF} = TF \cdot IDF$. <br>


As we can see TF-IDF is the combination of two operations TF and IDF. Let's implement this step by step.

In [14]:
#Open vocubalary.pkl
with open('vocabulary.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)
f.close()

In [15]:
import math

#init
tf = {}

#add keys to tf dictionary
for word in loaded_dict:
  tf[loaded_dict[word]] = {}

#idx = document_id, corpus = list of documents
for idx, corpus in enumerate(df['list_words'], start = 1):

  dim_corpus = len(corpus)
  for word in corpus:

    #compute tf
    if word in tf[loaded_dict[word]]:
      tf[loaded_dict[word]][idx] += 1/dim_corpus

    else:
      tf[loaded_dict[word]][idx] = 1/dim_corpus

#init idf
idf = {}
for i in tf:

  idf.update({i: math.log(len(df['list_words'])/len(tf[i]))})


#now, let's combine tf and idf
tf_idf = {}

#iterating over {key:value} pairs in tf
for (key1,value1) in tf.items():

  l = []
  #iterating over the value-dict of tf keys
  for key2, value2 in value1.items():
    l.append((key2, (value2*idf[key1])))

  tf_idf.update({key1 : l })
    

In [16]:
#save tf_idf into pickle file
with open('tf_idf.pkl', 'wb') as f:
  pickle.dump(tf_idf,f)
f.close

<function BufferedWriter.close>

### 2.2.2) Execute the query

**Cosine similarity** <br>
$\begin{equation} \cos{(d_j,q)} = \dfrac{\overrightarrow{d_j}\cdot \overrightarrow{q} }{||\overrightarrow{d_j}||\cdot ||\overrightarrow{q}||}=\dfrac{\sum\limits_{i=1}^nw_{i,j} \cdot w_{i,q}}{\sqrt{\sum\limits_{i=1}^nw_{i,j}^2}\cdot \sqrt{\sum\limits_{i=1}^nw_{i,q}^2} }\end{equation}$

In [1]:
#Cosine similarity function
import numpy as np
from numpy.linalg import norm

def cos_sim(query, doc):
  scal_prod = np.dot(query,doc)
  cosine_s = scal_prod/(norm(query)*norm(doc))
  return cosine_s

In [2]:
#ask the user the query
query = input('please insert a query: \n')
query_words = query.split()  #return a list of strings

please insert a query: 
american museum


In [17]:
#pre-processing of the query
cleaned_query = pre_processing(query)

In [18]:
#We create the query vector for which we have computed the related tf_idf
tf_query = 1 / len(cleaned_query)
query_vec = []
for word in query_words:

  item_id = loaded_dict[word]
  query_vec.append(tf_query * idf[item_id])


In [19]:
#retrieve term_id from the query
term_id_query = []
for i in cleaned_query:
  term_id_query.append(loaded_dict[i])


In [20]:
#retrieve a list with the documents containining query words
results = [set(tf_idf[item]) for item in [loaded_dict[word] for word in cleaned_query]]

In [21]:
#obtain vectors for the docs
all_match = {}

for word in query_words:       #slide over the query word
  item_id = loaded_dict[word]  #retrieve the item id of the query word
  #print(word)
  for r in results:   #slide over the tuples in the list of results
    for doc in r:
      if doc[0] not in all_match.keys():

        all_match[doc[0]] = [float(doc[1])]
        #print(type(all_match[doc[0]]))
      else: 
        if len(all_match[doc[0]]) < len(query_words):
          all_match[doc[0]].append(doc[1])
        
        

In [22]:
#we want in the dataframe all the docs that contain all the query words
list_doc = []
for doc in all_match.keys():
  list_doc.append(doc)
docs = df.filter(items = list_doc, axis = 0)
docs = docs[['placeName', 'placeDesc', 'placeURL']]

In [23]:
#now we have the vectorial representation of the documents. 
#What we have to do now is computing cosìne similariyìty between the query vector and every document containing all the query word. 

for k, doc in all_match.items():
  
  docs.loc[k,'Similarity'] = cos_sim(query_vec, doc)
  


In [26]:
#retrieve the first k=10 documents
final_df = docs.sort_values(by=['Similarity'], ascending=False).head(10)

In [27]:
final_df

,placeName,placeDesc,placeURL,Similarity
1707,Salina Turda,"With over 2,000 years of history, highly prese...",https://www.atlasobscura.com/places/salina-turda,1.0
6038,World's Heaviest Ball of Twine,The United States is filled with controversies...,https://www.atlasobscura.com/places/jfk-ball-o...,1.0
1813,Wanamaker Building,John Wanamaker did things a bit differently. I...,https://www.atlasobscura.com/places/wanamaker-...,1.0
1068,Chand Baori,"In 1864, the famous French world traveler Loui...",https://www.atlasobscura.com/places/ancient-st...,1.0
2018,Carroll Avenue,At the mouth of the north branch of the Chicag...,https://www.atlasobscura.com/places/carroll-av...,1.0
5807,Indian Village of the Ahwahnee,Behind the Visitor Center near the Yosemite Mu...,https://www.atlasobscura.com/places/indian-vil...,1.0
1076,The Cartoon Museum,"It might surprise the characters inside, but t...",https://www.atlasobscura.com/places/the-cartoo...,1.0
4951,Texas Triffid Ranch,Billing itself as “Dallas’s pretty much only c...,https://www.atlasobscura.com/places/texas-trif...,1.0
3603,Silverball Museum Arcade,"A retro hub of flashing bulbs, mechanical bell...",https://www.atlasobscura.com/places/silverball...,1.0
1850,St. Beatus Cave,The legend of the cave revolves around its nam...,https://www.atlasobscura.com/places/st-beatus-...,1.0
